## 1) First, do some data preprocessing to clean up the data as necessary.

In [1]:
!pip install markovify --quiet

In [2]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import nltk
import spacy
import re
import markovify

In [3]:
postgres_user = "dsbc_student"
postgres_pw = "7*.8G9QH21"
postgres_host = "142.93.121.174"
postgres_port = "5432"
postgres_db = "twitter_sentiment"

engine = create_engine(
    "postgresql://{}:{}@{}:{}/{}".format(
        postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db
    )
)

twitter = pd.read_sql_query("select * from twitter", con=engine)

# no need for an open connection, as we're only doing a single query
engine.dispose()


twitter.head()

/usr/local/lib/python3.6/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


,index,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,0,570306133677760513,neutral,1.0000,None,NaN,Virgin America,None,cairdin,None,0,@VirginAmerica What @dhepburn said.,None,2015-02-24 11:35:52 -0800,None,Eastern Time (US & Canada)
1,1,570301130888122368,positive,0.3486,None,0.0000,Virgin America,None,jnardino,None,0,@VirginAmerica plus you've added commercials t...,None,2015-02-24 11:15:59 -0800,None,Pacific Time (US & Canada)
2,2,570301083672813571,neutral,0.6837,None,NaN,Virgin America,None,yvonnalynn,None,0,@VirginAmerica I didn't today... Must mean I n...,None,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,None,jnardino,None,0,@VirginAmerica it's really aggressive to blast...,None,2015-02-24 11:15:36 -0800,None,Pacific Time (US & Canada)
4,4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,None,jnardino,None,0,@VirginAmerica and it's a really big bad thing...,None,2015-02-24 11:14:45 -0800,None,Pacific Time (US & Canada)


In [4]:
twitter = pd.DataFrame(twitter)
twitter.head()

,index,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,0,570306133677760513,neutral,1.0000,None,NaN,Virgin America,None,cairdin,None,0,@VirginAmerica What @dhepburn said.,None,2015-02-24 11:35:52 -0800,None,Eastern Time (US & Canada)
1,1,570301130888122368,positive,0.3486,None,0.0000,Virgin America,None,jnardino,None,0,@VirginAmerica plus you've added commercials t...,None,2015-02-24 11:15:59 -0800,None,Pacific Time (US & Canada)
2,2,570301083672813571,neutral,0.6837,None,NaN,Virgin America,None,yvonnalynn,None,0,@VirginAmerica I didn't today... Must mean I n...,None,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,None,jnardino,None,0,@VirginAmerica it's really aggressive to blast...,None,2015-02-24 11:15:36 -0800,None,Pacific Time (US & Canada)
4,4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,None,jnardino,None,0,@VirginAmerica and it's a really big bad thing...,None,2015-02-24 11:14:45 -0800,None,Pacific Time (US & Canada)


In [5]:
# Utility function for standard text cleaning
def text_cleaner(text):
    type(text)
    text = re.sub(r"--", " ", text)
    text = re.sub("[\[].*?[\]]", "", text)
    text = re.sub(r"(\b|\s+\-?|^\-?)(\d+|\d*\.\d+)\b", " ", text)
    text = " ".join(text.split())
    return text

In [6]:
tweet_text = []

for tweet in twitter["text"]:
    cleaned = text_cleaner(tweet)
    tweet_text.append(cleaned)

tweets = pd.DataFrame(tweet_text, columns=["cleaned_text"])
tweets.head()

,cleaned_text
0,@VirginAmerica What @dhepburn said.
1,@VirginAmerica plus you've added commercials t...
2,@VirginAmerica I didn't today... Must mean I n...
3,@VirginAmerica it's really aggressive to blast...
4,@VirginAmerica and it's a really big bad thing...


In [7]:
twitter['text'] = tweets

In [8]:
twitter.head()

,index,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,0,570306133677760513,neutral,1.0000,None,NaN,Virgin America,None,cairdin,None,0,@VirginAmerica What @dhepburn said.,None,2015-02-24 11:35:52 -0800,None,Eastern Time (US & Canada)
1,1,570301130888122368,positive,0.3486,None,0.0000,Virgin America,None,jnardino,None,0,@VirginAmerica plus you've added commercials t...,None,2015-02-24 11:15:59 -0800,None,Pacific Time (US & Canada)
2,2,570301083672813571,neutral,0.6837,None,NaN,Virgin America,None,yvonnalynn,None,0,@VirginAmerica I didn't today... Must mean I n...,None,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,None,jnardino,None,0,@VirginAmerica it's really aggressive to blast...,None,2015-02-24 11:15:36 -0800,None,Pacific Time (US & Canada)
4,4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,None,jnardino,None,0,@VirginAmerica and it's a really big bad thing...,None,2015-02-24 11:14:45 -0800,None,Pacific Time (US & Canada)


In [9]:
nlp = spacy.load("en", disable=["parser", "ner"])
nlp.add_pipe(nlp.create_pipe("sentencizer"))

# below is necessary to avoid memory error of SpaCy
nlp.max_length = 20000000
doc = nlp(" ".join(twitter["text"]))

In [10]:
print("The doc object is a {} object.".format(type(doc)))
print("It is {} tokens long".format(len(doc)))
print("The first three tokens are '{}'".format(doc[:3]))
print("The type of each token is {}".format(type(doc[0])))

The doc object is a <class 'spacy.tokens.doc.Doc'> object.
It is 300214 tokens long
The first three tokens are '@VirginAmerica What @dhepburn'
The type of each token is <class 'spacy.tokens.token.Token'>


In [11]:
# make sentences of more than 2 characters
sents = [sent.text for sent in doc.sents if len(sent.text) > 2]

In [12]:
sents[:5]

['@VirginAmerica What @dhepburn said.',
 "@VirginAmerica plus you've added commercials to the experience... tacky.",
 "@VirginAmerica I didn't today... Must mean I need to take another trip!",
 '@VirginAmerica it\'s really aggressive to blast obnoxious "entertainment" in your guests\' faces &amp; they have little recourse @VirginAmerica and it\'s a really big bad thing about it @VirginAmerica seriously would pay $ a flight for seats that didn\'t have this playing.',
 "it's really the only bad thing about flying VA @VirginAmerica yes, nearly every time I fly VX this “ear worm” won’t go away :) @VirginAmerica Really missed a prime opportunity for Men Without Hats parody, there."]

## 2) Train a Markov chain model by using only the negative sentiment tweets. Generate some random sentences. Do the generated sentences exhibit the same negative sentiment?

In [18]:
#make negative tweets doc
twitter_negative_doc = nlp(" ".join(twitter[twitter["airline_sentiment"]=="negative"].text))

# make sentences of more than 2 characters
tweet_negative_sents = " ".join([sent.text for sent in twitter_negative_doc.sents if len(sent.text) > 2])

#instantiate markovify
tweet_negative_generator = markovify.Text(tweet_negative_sents, state_size = 3)

#10 randomly generated negative sentences with max 150 characters
for i in range(10):
    print(tweet_negative_generator.make_sentence(max_chars=150, tries=100))
    print('----------------------------------------------------------------')

@USAirways Can't stress enough how awful the agent was for flight . I cant imagine it's on time on your website it says days.
----------------------------------------------------------------
@united Classic @united flight UA3364 been sitting on this plane for an hour.
----------------------------------------------------------------
@USAirways over hours and it automatically hangs up on me after I called United they said they can't help me and it's out of their hands.
----------------------------------------------------------------
# GetMartyHome @united has not responded to my tweets and DM??? Really sad feeling to be ignored.
----------------------------------------------------------------
How can you not put my bag on due to your own making.
----------------------------------------------------------------
But it's unfair to stay more than h traveling @united if I pay you $ and $ for my luggage on flight . What to do?
----------------------------------------------------------------
@u

## 3) Repeat the previous task, but this time, use only the positive sentiment tweets. Generate some random sentences and observe whether they exhibit positive sentiment or not.

In [20]:
#make negative tweets doc
twitter_positive_doc = nlp(" ".join(twitter[twitter["airline_sentiment"]=="positive"].text))

# make sentences of more than 2 characters
tweet_positive_sents = " ".join([sent.text for sent in twitter_positive_doc.sents if len(sent.text) > 2])

#instantiate markovify
tweet_positive_generator = markovify.Text(tweet_positive_sents, state_size = 3)

#10 randomly generated positve sentences with max 150 characters
for i in range(10):
    print(tweet_positive_generator.make_sentence(max_chars=150, tries=100))
    print('----------------------------------------------------------------')

# oscars2016 .@SouthwestAir Thx for the quick response!
----------------------------------------------------------------
@AmericanAir thank you for listening to me vent.
----------------------------------------------------------------
Now on the plane now!
----------------------------------------------------------------
@JetBlue had a great flight up to Vancouver.
----------------------------------------------------------------
Got me to my destination @USAirways thanks for your reply.
----------------------------------------------------------------
@AmericanAir will award me , air miles!!! Yes I am going to take an hour to type it out @AmericanAir @beantownmatty Sounds like a date!
----------------------------------------------------------------
Thanks again @USAirways on time today @AmericanAir thanks.
----------------------------------------------------------------
@JetBlue Thanks for the reply and the follow.
----------------------------------------------------------------
Used it 

## 4) Now, train your Markov chain model using all of the tweets. Generate some random sentences. Can you say something systematic about the sentiments of the generated tweets?

In [25]:
airline_tweets_generator = markovify.Text(sents, state_size=3)

#10 randomly generated sentences with max 150 characters
for i in range(10):
    print(airline_tweets_generator.make_sentence(max_chars=150, tries=100))
    print('----------------------------------------------------------------')

@AmericanAir Need to change flight, but the CS # is telling me no refund for unused portion of tix.
----------------------------------------------------------------
sucks @USAirways ...these changes as well to Late Flightr in the day would be super helpful.
----------------------------------------------------------------
I was supposed to depart at : @USAirways hours and counting @united DM'd you hrs ago at your request.
----------------------------------------------------------------
@SouthwestAir too long to wait on hold and issue still not resolved.
----------------------------------------------------------------
@VirginAmerica would love to go to get it.
----------------------------------------------------------------
FAIL @united you are trying to find another???
----------------------------------------------------------------
I've called them times today and not once was I able to talk so someone so had to go with #CarrieUnderwood 😍👌 @VirginAmerica @LadyGaga @CarrieUnderwood Sorr

A lot of the tweets seem to lean towards a more negative general sentiment. These 10 randomly generated tweets seem to focus on a person venting frustration about their flight or asking for help with their flight arrangements. 